# Import Library

In [1]:
# data download from kaggle
import kagglehub
# data manipulation
import numpy as np
import pandas as pd
# extracting url
import re
# for KNN fill null
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

# Load Dataset

In [2]:
# dataset is downloaded from https://www.kaggle.com/datasets/andrewmvd/okcupid-profiles
path = kagglehub.dataset_download("andrewmvd/okcupid-profiles")

In [ ]:
# load csv from the just downloaded dataset
df = pd.read_csv(path + "/okcupid_profiles.csv")
# load csv if you have already downloaded the dataset
# df = pd.read_csv("okcupid_profiles.csv")

# Simple Investigation

In [4]:
# Check top 5 rows of dataset
df.head()

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",...,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...
1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,white,...,i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,NaN,NaN,i am very open and will share just about anyth...,NaN
2,38,available,m,straight,thin,anything,socially,NaN,graduated from masters program,NaN,...,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement conversation creation contemplation t...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ..."
3,23,single,m,straight,thin,vegetarian,socially,NaN,working on college/university,white,...,i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . . lynch, jarmusch...",NaN,cats and german philosophy,NaN,NaN,you feel so inclined.
4,29,single,m,straight,athletic,NaN,socially,never,graduated from college/university,"asian, black, other",...,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at: http://bagsbrown....,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians at the moment...",NaN,NaN,NaN,NaN,NaN


In [5]:
# check column name, data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59946 entries, 0 to 59945
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          59946 non-null  int64  
 1   status       59946 non-null  object 
 2   sex          59946 non-null  object 
 3   orientation  59946 non-null  object 
 4   body_type    54650 non-null  object 
 5   diet         35551 non-null  object 
 6   drinks       56961 non-null  object 
 7   drugs        45866 non-null  object 
 8   education    53318 non-null  object 
 9   ethnicity    54266 non-null  object 
 10  height       59943 non-null  float64
 11  income       59946 non-null  int64  
 12  job          51748 non-null  object 
 13  last_online  59946 non-null  object 
 14  location     59946 non-null  object 
 15  offspring    24385 non-null  object 
 16  pets         40025 non-null  object 
 17  religion     39720 non-null  object 
 18  sign         48890 non-null  object 
 19  smok

In [6]:
# check numerical data 
df.describe()

,age,height,income
count,59946.000000,59943.000000,59946.000000
mean,32.340290,68.295281,20033.222534
std,9.452779,3.994803,97346.192104
min,18.000000,1.000000,-1.000000
25%,26.000000,66.000000,-1.000000
50%,30.000000,68.000000,-1.000000
75%,37.000000,71.000000,-1.000000
max,110.000000,95.000000,1000000.000000


In [7]:
# check null value count
df.isnull().sum()

age                0
status             0
sex                0
orientation        0
body_type       5296
diet           24395
drinks          2985
drugs          14080
education       6628
ethnicity       5680
height             3
income             0
job             8198
last_online        0
location           0
offspring      35561
pets           19921
religion       20226
sign           11056
smokes          5512
speaks            50
essay0          5488
essay1          7572
essay2          9638
essay3         11476
essay4         10537
essay5         10850
essay6         13771
essay7         12451
essay8         19225
essay9         12603
dtype: int64

# Data Cleanse

## Transform Data

In [8]:
def process_sex_height(df):
    """"
    This function will convert
    - 'm' to male and 'f' to female
    - height from inches to cm
    """
    # make a copy of the dataframe
    df = df.copy()
    # convert 'm' to 'male' and 'f' to 'female'
    df['sex'] = df['sex'].map({'m': 'male', 'f': 'female'})
    # convert height from inches to cm
    df['height'] = round(df['height'] * 2.54)
    return df

## Fill Null by Meaningful text

In [9]:
def fill_missing_values(df):
    """
    This function will fill missing values with meaningful text, combine essays into one column and drop columns that are not needed

    """
    # make a copy of the dataframe
    df = df.copy()
    # fill missing values with meaningful text
    df['diet'] = df['diet'].fillna("anything")
    df['drinks'] = df['drinks'].fillna("not at all")
    df['drugs'] = df['drugs'].fillna("never")
    df['height'] = df['height'].fillna(df['height'].median())
    df['job'] = df['job'].fillna('other')
    df['offspring'] = df['offspring'].fillna("no kids and neutral to kids")
    df['pets'] = df['pets'].fillna("no pets and neutral to pets")
    df['religion'] = df['religion'].fillna("irreligion")
    df['sign'] = df['sign'].str.replace("&rsquo;", "'").fillna("unknown zodiac sign")
    df['smokes'] = df['smokes'].fillna("no")
    df['speaks'] = df['speaks'].fillna("english")
    # filling missing values for essays, combine essays into one column and remove excess ","
    df['essay_all'] = df.loc[:,"essay0":"essay9"].fillna("").apply(lambda x: ','.join(x.astype(str)),axis=1).str.strip(",").fillna("")
    # drop columns that are not needed
    columns_to_drop = ['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6','essay7', 'essay8', 'essay9', 'last_online']
    df = df.drop(columns=columns_to_drop)
    
    return df

## Fill Value by KNN

In [10]:
def fill_missing_values_knn(df, col_fill, list_col_ref, list_col_num):
    """
    This function will fill missing values using KNN imputer
    """

    # Copy dataframe to avoid modifying original dataframe
    df = df.copy()

    # Create a label encoder object
    label_encoder = LabelEncoder()

    # drop rows with missing values in the column to be imputed
    df_non_null = df[col_fill].dropna()
    # Fit and transform the column to be imputed
    df.loc[df_non_null.index, col_fill +'_encoded'] = label_encoder.fit_transform(df_non_null)

    # Convert the column to float
    df[col_fill + '_encoded'] = df[col_fill + '_encoded'].astype(float)
    # Fill missing values with NaN
    df.loc[df[col_fill].isnull(), col_fill + '_encoded'] = np.nan

    # Print the mapping of the label encoder
    print("Label Encoding Mapping: ", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

    # Convert categorical variables into numerical format
    for col in list_col_ref:
        df[col + '_encoded'] = LabelEncoder().fit_transform(df[col].astype(str))  # Encode as numbers

    # List of features to be used for KNN including numerical columns, encoded categorical columns and encoded column to be imputed
    knn_features = list_col_num + [col + '_encoded' for col in list_col_ref] + [col_fill + '_encoded']

    # Ensure only features to be used for KNN are selected
    df_knn = df[knn_features]

    # Initialize KNN Imputer with 5 neighbors
    imputer = KNNImputer(n_neighbors=5, weights='distance')
    # Fit and transform the KNN imputer
    df_knn_imputed = imputer.fit_transform(df_knn)

    # Replace original dataframe values with imputed values
    df[knn_features] = df_knn_imputed

    # Round to nearest integer before inverse transforming
    df[col_fill + '_encoded'] = df[col_fill + '_encoded'].round().astype(int)

    # Inverse transform the encoded column to get the original values
    df[col_fill + '_imputed'] = label_encoder.inverse_transform(df[col_fill + '_encoded'])

    # Replace the original column with the imputed values
    df[col_fill] = df[col_fill + '_imputed']

    # Drop columns that are no longer needed
    df = df.drop(columns=[col_fill + '_encoded', col_fill + '_imputed'] + [col + '_encoded' for col in list_col_ref])

    return df

def fill_missing_values_knn_multi_col(df, col_fillna):
    """
    This function will fill missing values using KNN imputer for multiple columns
    """
    # Copy dataframe to avoid modifying original dataframe
    df = df.copy()
    # fill missing values for each key in the dictionary
    for col_fill, list_col_ref in col_fillna.items():
        df = fill_missing_values_knn(df, col_fill, list_col_ref[0], list_col_ref[1])
    return df

## Extract and Remove URL

In [11]:
def extract_urls(text):
    """
    This function will extract URLs from text for one row
    """
    # Check if text is a string
    if not isinstance(text, str):
        return []
    # extract URLs
    url_pattern = r'\b(?=https?:\/\/|www\.[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6})\b([-a-zA-Z0-9(@:%_\+.~#?&//=]*)'
    # Extract matches
    matches = re.findall(url_pattern, text)
    return matches

def remove_url(row):
    """"
    This function will remove URL from text for one row
    """
    # Get essay text
    essay = row['essay_all']
    # Remove URL from text
    for url in row['url']:
        essay = essay.replace(url, '')
    return essay

def remove_url_col(df):
    """
    This function will remove URL from text for each row in the dataframe
    """
    # Copy dataframe to avoid modifying original dataframe
    df = df.copy()
    # Extract URLs for each row
    df['url'] = df['essay_all'].apply(extract_urls)
    # Remove URLs for each row
    df['essay_all'] = df.apply(remove_url, axis=1)
    return df

## Combine Cleanse Data Process

In [12]:
def data_cleaning(df, col_fillna):
    """
    This function will clean the dataset by:
    - Converting
        - 'm' to 'male'
        - 'f' to 'female'
        - height from inches to cm 
    - Filling missing values with meaningful text
    - Filling missing values using KNN imputer
    - Removing URL from text
    """
    # Copy dataframe to avoid modifying original dataframe
    df = df.copy()
    df = process_sex_height(df)
    df = fill_missing_values(df)
    df = fill_missing_values_knn_multi_col(df, col_fillna)
    df = remove_url_col(df)
    # Drop columns that are not needed
    df = df.drop(columns=['income','url'])
    return df

# Perform Data Cleansing

In [13]:
# Dictionary of columns to fillna as key, list of columns to be used as reference and numerical columns as value
col_fillna = {'body_type': [['sex', 'drinks', 'diet', 'drugs'], ['age', 'height']],
              'education': [['job', 'location'], ['age', 'income']],
              'ethnicity': [['religion', 'location'], ['age']]}

In [14]:
# Call the data_cleaning function
df_cleaned = data_cleaning(df, col_fillna)

Label Encoding Mapping:  {'a little extra': np.int64(0), 'athletic': np.int64(1), 'average': np.int64(2), 'curvy': np.int64(3), 'fit': np.int64(4), 'full figured': np.int64(5), 'jacked': np.int64(6), 'overweight': np.int64(7), 'rather not say': np.int64(8), 'skinny': np.int64(9), 'thin': np.int64(10), 'used up': np.int64(11)}
Label Encoding Mapping:  {'college/university': np.int64(0), 'dropped out of college/university': np.int64(1), 'dropped out of high school': np.int64(2), 'dropped out of law school': np.int64(3), 'dropped out of masters program': np.int64(4), 'dropped out of med school': np.int64(5), 'dropped out of ph.d program': np.int64(6), 'dropped out of space camp': np.int64(7), 'dropped out of two-year college': np.int64(8), 'graduated from college/university': np.int64(9), 'graduated from high school': np.int64(10), 'graduated from law school': np.int64(11), 'graduated from masters program': np.int64(12), 'graduated from med school': np.int64(13), 'graduated from ph.d prog

In [15]:
# Check top 5 rows of cleaned dataset
df_cleaned.head()

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,height,job,location,offspring,pets,religion,sign,smokes,speaks,essay_all
0,22.0,single,male,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",190.0,transportation,"south san francisco, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism and very serious about it,gemini,sometimes,english,about me: i would love to think that i was so...
1,35.0,single,male,straight,average,mostly other,often,sometimes,working on space camp,white,178.0,hospitality / travel,"oakland, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism but not too serious about it,cancer,no,"english (fluently), spanish (poorly), french (...",i am a chef: this is what that means. 1. i am ...
2,38.0,available,male,straight,thin,anything,socially,never,graduated from masters program,"black, native american, pacific islander, white",173.0,other,"san francisco, california",no kids and neutral to kids,has cats,irreligion,pisces but it doesn't matter,no,"english, french, c++","i'm not ashamed of much, but writing public te..."
3,23.0,single,male,straight,thin,vegetarian,socially,never,working on college/university,white,180.0,student,"berkeley, california",doesn't want kids,likes cats,irreligion,pisces,no,"english, german (poorly)","i work in a library and go to school. . .,read..."
4,29.0,single,male,straight,athletic,anything,socially,never,graduated from college/university,"asian, black, other",168.0,artistic / musical / writer,"san francisco, california",no kids and neutral to kids,likes dogs and likes cats,irreligion,aquarius,no,english,hey how's it going? currently vague on the pro...


# Save Cleansed Dataset as .csv

In [16]:
# save cleaned dataset as csv
df_cleaned.to_csv("okcupid_profiles_cleaned.csv", index=False)

In [17]:
# check if the cleaned dataset is saved correctly
pd.read_csv("okcupid_profiles_cleaned.csv").fillna("").equals(df_cleaned)

True